In [2]:
import numpy as np 
import matplotlib.pyplot as plt


class stacking(object):
    """
    stacking 0-1re
    S/N>5 sigma
    """
    def __init__(self,plateifu,wave,flux,ivar,flux_map,ivar_map,mask,ellcoo,v,plot=True):
        self.plateifu=plateifu
        self.flux = flux
        self.wave = wave
        self.ivar=ivar
        self.flux_map=flux_map
        self.ivar_map=ivar_map
        self.mask=mask
        self.ellcoo = ellcoo
        self.v=v
        
        self.pre=self.remove_agn_badspx_ellcoo()
        if self.pre>0.3:
            self.vel2z()
            self.stack()
                
    def remove_agn_badspx_ellcoo(self):
        """
        remove bad spaxel in map data
        """
        mask_bad=(self.mask!=0) # !=0 remove
        
        """
        BPT diagram remove AGN
        Kauffmann 2003: log(OIII/hb)>0.61/(log(NII/Ha)-0.05)+1.13
        [oiii]5008,4960:13,12(5007)
        Hb4862:11
        NII6549,6585:17,19(6584)
        Ha6564:18
        """        
        x=np.log10(self.flux_map[19]/self.flux_map[18])
        y=np.log10(self.flux_map[13]/self.flux_map[11])
        mask_bpt=(y<=0.61/(x-0.05)+1.13)
        mask_bpt_vaild=(y>0.61/(x-0.05)+1.13)
        #mask in agn is not same with sf
        #f=np.ma.array(f,mask=mask)
        
        """
        ellcoo
        """
        mask_ellcoo=(self.ellcoo>1)
        
        mask=mask_bad|mask_bpt|mask_ellcoo
        
        self.flux_map=np.ma.array(self.flux_map,mask=mask)
        self.ivar_map=np.ma.array(self.ivar_map,mask=mask)
        self.flux=np.ma.array(self.flux,mask=mask)
        self.ivar=np.ma.array(self.ivar,mask=mask)
        self.v=np.ma.array(self.v,mask=mask)
        
        """
        count effective points
        """
        mask_1=(self.ellcoo<=1)
        num_all=np.sum(mask_1)
        mask_2=mask_1&(mask_bpt_vaild)&(self.mask==0)
        num_valid=np.sum(num_vaild)
        pre=num_vaild/num_all
        print("percentage of vaild points : %s / %s"%(num_vaild,num_all))
        print("percentage of vaild points : %s",num_vaild/num_all)
        
        return pre
                
    def sf(self):
        """
        Pick out SF region:
        """
        x=np.log10(self.flux_map[19]/self.flux_map[18])
        y=np.log10(self.flux_map[13]/self.flux_map[11])
        mask=(y<0.61/(x-0.05)+1.13)
        # f=f[mask]
        return mask
        
    
    def vel2z(self):
        self.z=self.v/(3e5)

        
    def stack(self):
        """
        stacking the spectrum
        """
        print('self.wave.shape[0]',self.wave.shape[0])
        self.flux_stack=np.zeros(self.wave.shape[0])
        self.ivar_stack=np.zeros(self.wave.shape[0])
        for i in range(0,len(self.flux),1):
            flux=self.flux[i]
            ivar=self.ivar[i]
            z=self.z[i]
            wave_rest=self.wave/(1+z)
            flux=np.interp(wave,wave_rest,flux)
            ivar=np.interp(wave,wave_rest,ivar)
            # remove ivar==0:
            if flux.shape[0]>0:
                # shape not size
                mask=(ivar==0)
                flux=np.ma.array(flux,mask=mask)
                ivar=np.ma.array(ivar,mask=mask)
                self.flux_stack+=flux
                self.ivar_stack+=ivar
                # ivar=sum(ivar)            
        """
        SNR
        """
        snr=np.sum(self.flux_stack)*np.sqrt(np.sum(self.ivar_stack))
        if snr>5:
            
            print('plateifu',self.plateifu)
            table=[self.wave]
            table.append(self.flux_stack)
            table.append(self.ivar_stack)
            table=np.transpose(table)
            t=Table(table,names=['wave','flux','ivar'])
            t.write(self.dir+'%s_1re_stack.fits',format='fits')
        else:
            print('snr<5',self.plateifu)
            

SyntaxError: unexpected EOF while parsing (<ipython-input-2-854eee28136f>, line 42)

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import os
from astropy.io import fits
import numpy as np


import stack

np.seterr(divide='ignore', invalid='ignore')

dir_drp = '/Users/astro/Documents/notebooks/manga/spectro/redux/v2_5_3/'
dir_dap = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/HYB10-MILESHC-MILESHC/'
dir_res = '/Users/astro/Documents/notebooks/zs/2019-10-18/stacking/'


def stacking(plateifu):
    print(plateifu)
    plate = plateifu.split('-')[0]
    ifu = plateifu.split('-')[1]
    f_map = dir_dap+plate+'/'+ifu+'/manga-'+plate + \
        '-'+ifu+'-MAPS-HYB10-MILESHC-MILESHC.fits.gz'
    f_cube = dir_drp+plate+'/stack/manga-'+plate + \
        '-'+ifu+'-LOGCUBE.fits.gz'
    if os.path.exists(f_map) and os.path.exists(f_cube):
        cube = fits.open(f_cube)
        wave = cube['WAVE'].data
        flux_t = np.transpose(cube['FLUX'].data, axes=(1, 2, 0))
        ivar_t = np.transpose(cube['IVAR'].data, axes=(1, 2, 0))

        mapf = fits.open(f_map)
        flux_map = mapf['EMLINE_GFLUX'].data
        ivar_map = mapf['EMLINE_GFLUX_IVAR'].data
        mask_map = mapf['EMLINE_GFLUX_MASK'].data
        ellcoo = mapf['SPX_ELLCOO'].data[1]
        stellar_vel = mapf['STELLAR_VEL'].data

        stack.stack(plateifu, wave, flux_t, ivar_t, flux_map, ivar_map,
                          mask_map, ellcoo, stellar_vel,dir_res)


if __name__ == '__main__':
    dir_drpall = '/Users/astro/Documents/notebooks/manga/spectro/analysis/MPL-8/drpall-v2_5_3.fits'
    f = fits.open(dir_drpall)
    data = f[1].data
    plateifu = data.field('plateifu')
    print('              start                  :')
    for i in range(0,len(plateifu),1):
        stacking(plateifu[i])
    print('              finished               .')
#     print('the number of plateifu which have full manga data : ',i_havemanga)
# 


              start                  :
10001-12701
